In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
# 1. Path to the extracted dataset
data_dir = "archive/images"

# ✅ Check if folder exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Path does not exist: {data_dir}")

In [3]:
# 2. ImageDataGenerator with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [4]:
# 3. Model Architecture using EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
# 4. Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 5. Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

C:\Users\Tarun yadav\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
  83/2525 ━━━━━━━━━━━━━━━━━━━━ 2:18:55 3s/step - accuracy: 0.0648 - loss: 4.3898

In [ ]:
# 6. Calorie Dictionary (example)
calorie_dict = {
    "apple_pie": 296,
    "baby_back_ribs": 400,
    "baklava": 430,
    "caesar_salad": 190,
    "chicken_curry": 280,
    "samosa": 262,
    "gulab_jamun": 330,
    # Add more mappings based on class names
}

In [ ]:
# 7. Make predictions on validation set
val_generator.reset()
preds = model.predict(val_generator, verbose=1)
predicted_classes = np.argmax(preds, axis=1)
true_classes = val_generator.classes
class_labels = list(val_generator.class_indices.keys())

In [ ]:
# 8. Show a few predictions with estimated calories
for i in range(5):
    image, label = val_generator[i][0][0], val_generator[i][1][0]
    pred_class_index = np.argmax(preds[i])
    pred_class_name = class_labels[pred_class_index]
    calories = calorie_dict.get(pred_class_name, "Unknown")

    plt.imshow(image)
    plt.title(f"{pred_class_name} | Estimated Calories: {calories} kcal/100g")
    plt.axis('off')
    plt.show()